In [1]:
%cd ../

/u02/thanh/workplace/plasma


In [2]:
import plasma.functional as F

from plasma.meta.object_graph import ContextManager, Manager

/u02/thanh/workplace/plasma/.env/RnD/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class HUHU(F.AutoPipe):

    def __init__(self, name, age, test=5):
        super().__init__()
        self.name = name
        self.age = age
        self.test = test

In [4]:
class HEHE(F.AutoPipe):

    def __init__(self, huhu, bee) -> None:
        super().__init__()
        self.huhu = huhu

In [5]:
class HAHA(F.AutoPipe):

    def __init__(self, hehe:HEHE, huhu:HUHU, test_factory:dict) -> None:
        super().__init__()
        print(test_factory)
        self.hehe = hehe
        self.huhu = huhu

In [6]:
manager =  ContextManager()
manager

In [7]:
manager\
    .init_context('test')\
        .add_dependency('haha', HAHA)

AttributeError: 'MultiDiGraph' object has no attribute 'add_context'

In [ ]:
manager\
    .init_context('test2')\
    .add_dependency('huhu', HUHU)\
        .manager\
            .link(['test.huhu', 'test2.huhu'])
                